<a href="https://colab.research.google.com/github/JatinDutta0504/LearningBeam/blob/master/CompositeTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!{'pip install --quiet apache_beam'}

In [0]:
!{'mkdir -p data'}

In [0]:
from google.colab import files
uploaded = files.upload()

In [6]:
import apache_beam as beam

#create composite transform class from PTransform base class
class MyCompositeTransform(beam.PTransform):
  
  def expand(self,input_coll):
    a=(
        input_coll
            |beam.CombinePerKey(sum)
            |beam.Filter(filter_on_count)
    )
    return a

#filter all the dept count for > 30 only
def filter_on_count(element):

  name,count=element
  if count>30:
    return element

p1 = beam.Pipeline()

input_collection = (
                      p1
                        |'Read from file' >> beam.io.ReadFromText('dept_data.txt')
                        |'Split Rows' >> beam.Map(lambda record: record.split(','))
                    )

attendance_Accounts = (
                          input_collection
                          |beam.Filter(lambda record: record[3]=='Accounts')
                          |'Get the dept data' >> beam.Map(lambda record: ('Accounts ' + record[1],1))
                          |'Composite transform for Accounts' >> MyCompositeTransform() #call composite transform
                          #|beam.io.WriteToText('data/Accounts')
                      )

attendance_HR = (
                    input_collection
                    |beam.Filter(lambda record: record[3]=='HR')
                    |'Get the HR data' >> beam.Map(lambda record: ('HR ' + record[1],1))
                    |'Composite transform for HR' >> MyCompositeTransform() #call composite transform
                   # |beam.io.WriteToText('data/HR')
                )

output = (
          (attendance_Accounts,attendance_HR)
            |beam.Flatten()
            |beam.io.WriteToText('data/output')
        )
p1.run()


!{('head -n 20 data/output-00000-of-00001')}

('HR Beryl', 62)
('HR Olga', 31)
('HR Leslie', 31)
('HR Mindy', 31)
('HR Vicky', 31)
('HR Richard', 31)
('HR Kirk', 31)
('HR Kaori', 31)
('HR Oscar', 31)
('Accounts Marco', 31)
('Accounts Rebekah', 31)
('Accounts Itoe', 31)
('Accounts Edouard', 31)
('Accounts Kyle', 62)
('Accounts Kumiko', 31)
('Accounts Gaston', 31)
